# **LAB TWO: IMAGING**

This lab covers basic MR Imaging concepts: Gradients, Frequency Encoding, Phase Encoding, *k*-Space, Field of View, and Resolution.

> -------------------------------------------------------------------------------------------------------------------------------------------------------
> #### **Setup Task: Run the Notebook**
> 
> 1. Edit the cell below to set the `LAB_USER_NAME` variable to your name
> 2. Click **Run->Run All Cells** in the in top menu bar of jupyterlab
> 3. Open the Table of Contents side-bar on the left edge of jupyterlab to aid in navigation
> 
> -------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
LAB_USER_NAME = 'REPLACE_ME'

**Important**: To initialise this notebook, edit the cell above to set `LAB_USER_NAME` to your name, then click **Run->Run All Cells** in the top menu bar.

In [ ]:
import bokeh.plotting
bokeh.plotting.output_notebook(resources=bokeh.resources.INLINE) # use inline js to allow it to work offline
import panel as pn
pn.config.inline = True # use inline js to allow it to work offline
pn.extension(raw_css=['''progress {margin: 0;}''']) # raw_css setting is a workaround for panel issue 4112
import sys
import os
import numpy as np

# add inline dashboard libraries to path so they can be imported later
sys.path.append('../../../dashboards-inline')

LAB_DIR = os.path.join('/home/data/', LAB_USER_NAME)
os.makedirs(LAB_DIR, exist_ok=True)
print('User data directory:', LAB_DIR)

## 1. Gradients

<figure style="float: right;">
<img src="Images/GradCoilZ.png" width="400">
<figcaption style="width: 400px;">Figure 1.1: Z cradient coils and current directions (orange arrows). At $z_1$ the total field strength is increased, and at $z_2$ it is decreased.</figcaption>
</figure>

Part of Lab 1 involved optimising the duration of the NMR signal by correcting the magnetic field inhomogeneities (unevenness) using shims. Inhomogeneity causes nuclei at different locations in the sample to resonate at slightly different frequencies, resulting in destructive interference and a reduction in signal duration. The fundamental principle of NMR imaging is to exploit the effect of inhomogeneity to locate the source of an NMR signal in space. To purposely distort the field in a way that is simple to analyse, we will use combinations of coils that create linear **gradients** in the the field strength along the $x$, $y$, and $z$ axes (see Figure 1.1).

Because the gradients are linear, the Larmor frequency at any location will be a linear function of its coordinate along the gradient axis, e.g. when applying gradient $G_z$:

$$ f = f_0 + \bar\gamma \cdot G_z \cdot z \tag{1.1}$$

Where $f_0$ is the Larmor frequency when no gradient is applied, and $\bar\gamma$ is the gyromagnetic ratio ($\bar\gamma = 42.58 \times 10^{6} \ \mathrm{Hz}/\mathrm{T}$).

In the next task we will use the spin echo sequence shown below (Figure 1.2); first with the gradients set to the optimised shim values, and then modified to see how they can be used to encode position information into the NMR signal.

<figure>
<img src="Images/se_1.png" width="500">
<figcaption style="width: 500px;">Figure 1.2: Spin Echo pulse sequence with the Z shim shown.</figcaption>
</figure>


> ---
> #### **Task 1.1**
> 
> 1. Insert the thin layer sample (make sure to centre the sample using the depth gauge).
> 2. Reset shims and start experiment with "Run Loop".
> 3. Zoom in on the echo signal (from 6us to 14us).
> 4. Move the sample up and down. **Question:** What happens to the signal when it is moved too far from the centre?
> 5. Return the sample to roughly the centre (where the signal returns to full amplitude).
> 6. Increase the Z shim to its limit (1.0, or -1.0). **Question:** How does the signal shape change when the shim is offset?
> 7. Move the sample up and down a small amount, so that the magnitude is not significantly affected.  **Question:** What happens to the signal frequency?
> 8. Click abort to stop the experiment before moving on.
> ---

In [ ]:
# Experiment 1: Full acquisition SE with constant shim
# load global shims file
import yaml
from matipo import GLOBALS_DIR
SHIM_ORDER = ['shim_x', 'shim_y', 'shim_z']
SHIM_FILE = os.path.join(GLOBALS_DIR, 'shims.yaml')

with open(SHIM_FILE, 'r') as f:
    SHIM_INIT = yaml.safe_load(f)

# create shim inputs, using saved user shims as initial values
shim_inputs = {}
for shim_key in SHIM_ORDER:
    shim_name = shim_key.split('_')[-1].upper()
    shim_inputs[shim_key] = pn.widgets.FloatInput(name=shim_name+' shim', start=-1, end=1, step=0.01, value=round(SHIM_INIT[shim_key], 2), width=80)

# button to reset shim inputs to the saved values
reset_btn = pn.widgets.Button(name='Reset', button_type='primary', align='end', width=100)

def reset_shim_inputs(e):
    for shim_key in SHIM_ORDER:
        shim_inputs[shim_key].value = round(SHIM_INIT[shim_key], 2)

reset_btn.on_click(reset_shim_inputs)
    
from full_acq_SE import FullAcqSEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    t_echo=0.01,
    n_scans=1,
    n_samples=1500,
    t_dw=10e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.2
)

# add shim inputs
override_pars.update(shim_inputs)

# create dashboard app
exp1_app = FullAcqSEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True
)

exp1_app.plot.figure.height=400

# display layout
pn.Column(
    # echo_time,
    pn.Row(*([shim_inputs[shim_key] for shim_key in SHIM_ORDER]+[reset_btn])), # take shim inputs in order and concatenate reset button
    exp1_app.main(),
    sizing_mode='stretch_width'
)

## 2. Frequency Encoding
### Theory

<figure style="float: right;">
<img src="Images/WaterSampleAddition.png" width="500">
<figcaption style="width: 500px;">Figure 2.1: Frequency Encoding</figcaption>
</figure>

The experiments in **Task 1.1** showed how the frequency of the NMR signal from a small sample varies with position when a gradient is applied by offsetting a shim value. In fact, with calibration to determine the value of $G_z$, the frequency can be measured and the position calculated by rearranging Equation 1.1.

Now consider a sample made up of multiple small partitions of fluid; each partition will emit a signal with frequency depending on its position, but the measured signal will be the sum of all of them, a complex shape caused by constructive and destructive intereference. The frequencies present in the measured signal can be extracted using a Fourier Transform, revealing individual peaks associated with the partitions (See Figure 2.1).

In this example the partitions of water are separated, but the same logic can be applied to a continuous shape. We can imagine slicing the sample into multiple slices along the gradient axis, so that the signals from the slices are similar to the signals from the partitions in Figure 2.1. When the total signal is measured and processed with a Fourier transform, the magnitude of the resulting spectrum at any frequency will be proportional to the amount of sample in the slice that is precessing at that frequency. This means the spectrum is a 1D projection image of the sample.

Mathematically, the total signal $s(t)$ we measure in the rotating frame is proportional to the integral of the magnetization $m(z)$ multiplied by a phasor contributed by the frequency shift caused by $G_z$:

$$ \large s(t) \propto \int_z m(z) e^{-i \bar\gamma G_z z t} dz \tag{2.1}$$

With the substitution $k = \bar\gamma G_z t$, this equation has the same form as the inverse Fourier transform, and $m(z)$ can be written as the Fourier transform of $s(t)$:

$$ \large m(z) \propto \int_t s(t) e^{i \bar\gamma G_z z t} dt \tag{2.2}$$

### Pulse Sequence

<figure style="float: right;">
<img src="Images/se_2.png" width="500">
<figcaption style="width: 500px;">Figure 2.2: the Spin Echo sequence with a switched frequency encode gradient ($G_z$) and gated acquisition.</figcaption>
</figure>

For simplicity, the shim values and gradient values are generally set seperately and added together by the hardware. The gradient also does not need to be active for the entire pulse sequence, only during the acquisition time and for a short time between the $90^\circ$ and $180^\circ$ RF pulses – to satisfy a requirement for optimal spin echo formation where the area under the gradient between the $90^\circ$ and $180^\circ$ pulses must match the area under the gradient from the $180^\circ$ to the echo time ($T_E$). This optimised pulse sequence is shown in Figure 2.2, and will be used in the next experiment.

>---
>#### **Task 2.1**
>1. With the thin layer sample still inserted, start the experiment with "Run Loop".
>2. Increase the read gradient to 0.25.
>3. Move the sample up and down again, and see how the spectrum changes. **Question:** Does this match your observations in the previous experiment?
>4. Inspect the 1D Z phantom and, based on your observations with the thin layer sample, hand-draw a rough plot of the spectrum you would expect to see when it is inserted.
>5. Insert the 1D Z phantom (centred with the depth gauge). **Question:** Does the spectrum look as you expected?
>6. Adjust the gradient strength through its possible range and note how the spectrum "image" changes. **Question:** Comparing the image with 0.25 gradient and 0.5 gradient: Which has a larger FOV? Which has higher SNR? Which has finer spatial resolution? (refer to the Image Characteristics section below)
>7. Click abort to stop the experiment before moving on.
>---

In [ ]:
# Experiment 2: SE with adjustable Z read gradient showing signal and spectrum of echo
# read gradient input
# TODO: calibrate and use mT/m
read_z_grad = pn.widgets.FloatInput(name='Read Z Gradient', start=-1, end=1, step=0.01, value=0, width=200)
exp2_n_samples = pn.widgets.IntInput(name='Samples', start=20, end=400, step=10, value=200, width=200)
exp2_dwell_time = pn.widgets.IntInput(name='Dwell Time (µs)', start=2, end=20, step=2, value=10, width=200)

# TODO: oversample and use decimation before taking the FFT for flat filter profile
from SE import SEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=lambda: (0, 0, read_z_grad.value),
    t_echo=0.012,
    n_scans=1,
    n_samples=lambda: exp2_n_samples.value,
    t_dw=lambda: 1e-6*exp2_dwell_time.value, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.2
)

# create dashboard app
exp2_app = SEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=True,
    enable_run_loop=True,
    flat_filter=True
)

exp2_app.plot1.figure.height=400
exp2_app.plot2.figure.height=400

# display layout
pn.Column(
    pn.Row(read_z_grad, exp2_n_samples, exp2_dwell_time),
    exp2_app.main(),
    sizing_mode='stretch_width'
)

### Image Characteristics

<figure style="float: right;">
<img src="Images/fov_spatialResolution.png" width="500">
<figcaption style="width: 500px;">Figure 2.3: How Dwell Time (sample spacing) and Number of Samples affect FOV and Resolution, with a fixed frequency encode gradient amplitude.</figcaption>
</figure>

Three important characteristics an image are:
- **Field of View (FOV)**: The physical size of the image, i.e. the size of the largest object that can be viewed without being cropped or wrapped.
- **Spatial Resolution**: The level of detail, usually expressed as a minimum distance between features such that they are still resolved, and not merging into a single blob. Note that this is not the same as the number of pixels in the image (Pixel Resolution). e.g. a spatial resolution of 1 mm is *finer resolution* than a spatial resolution of 2 mm.
- **Signal to Noise Ratio (SNR)**: The ratio of the amplitude of the signal in areas of image corresponding to the nuclei in the sample to the amplitude of the noise in areas without signal. A larger SNR is better.

The relationship of FOV and resolution to the pulse sequence parameters, dwell time and number of samples, can be unintuitive. Figure 2.3 shows how, compared to (a): (b) increasing the number of samples with the same dwell time does not affect FOV and results in finer resolution; (c) increasing the number of samples while decreasing the dwell time to keep the total acquisition time the same increases the FOV without affecting resolution, and (d) increasing dwell time without changing the number of samples decreases FOV and gives finer resolution (zoom in).

## 3. Projection Reconstruction

With the above method of creating a 1D projection image, a 2D image can be reconstructed by combining multiple images with the sample rotated at different angles. This is the method used to produce 3D images from X-ray projections, called Computed Tomography (with the X-ray machine rotated around the human). If we apply a combination of linear gradients they will add together to produce a net gradient in an oblique (off-axis) direction. Using this method the gradient can be rotated instead of the rotating the sample.

In the next experiment, the read gradient will be specified with a magnitude $|G|$ and angle $\theta$, and the $x$ and $y$ components will be automatically calculated using:

$$
G_x = |G| \cdot \cos \theta \\
G_y = |G| \cdot \sin \theta
\tag{3.1}
$$

<figure style="float: right;">
<img src="Images/projectionReconstruction.png" width="500">
<figcaption style="width: 500px;">Figure 3.1: Projection Reconstruction</figcaption>
</figure>

>---
>#### **Task 3.1**
>1. Insert the mystery sample at the correct depth and start the experiment below with "Run Loop".
>2. Increase the magnitude to expand the 1D image and try both physically rotating the sample and rotating the read gradient direction.
>3. Set the gradient magnitude appropriately so that the 1D image is not cropped with any gradient direction.
>4. Based on the 1D views at different angles, draw a 2D top-down diagram of where you think the water is located in the mystery sample. Hint: draw a square in your notebook, and for each edge draw the 1D image you get from orienting the gradient along that edge. Based on this you can draw inside the square where the water is located (See Figure 3.1).
>5. Click abort to stop the experiment before moving on.
>---

In [ ]:
# Experiment 3: 1D SE with adjustable X/Y vector read gradient
# read gradient input
# TODO: calibrate and use mT/m
exp3_read_grad_mag = pn.widgets.FloatInput(name='Read Gradient Magnitude', start=0, end=1, step=0.01, value=0, width=200)
exp3_read_grad_dir = pn.widgets.FloatInput(name='Read Gradient Direction (degrees)', start=0, end=360, step=10, value=0, width=200)
exp3_read_x_grad = pn.widgets.FloatInput(name='Read Gradient X Component', start=-1, end=1, step=0.01, value=0, width=200, disabled=True)
exp3_read_y_grad = pn.widgets.FloatInput(name='Read Gradient Y Component', start=-1, end=1, step=0.01, value=0, width=200, disabled=True)

def exp3_read_grad_cb(*events):
    exp3_read_x_grad.value = exp3_read_grad_mag.value*np.cos((np.pi/180)*exp3_read_grad_dir.value)
    exp3_read_y_grad.value = exp3_read_grad_mag.value*np.sin((np.pi/180)*exp3_read_grad_dir.value)

exp3_read_grad_mag.param.watch(exp3_read_grad_cb, ['value'], onlychanged=True)
exp3_read_grad_dir.param.watch(exp3_read_grad_cb, ['value'], onlychanged=True)

# TODO: oversample and use decimation before taking the FFT for flat filter profile
from SE import SEApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=lambda: (exp3_read_grad_mag.value*np.cos((np.pi/180)*exp3_read_grad_dir.value), exp3_read_grad_mag.value*np.sin((np.pi/180)*exp3_read_grad_dir.value), 0),
    t_echo=0.012,
    n_scans=2,
    n_samples=100,
    t_dw=20e-6, # using a long dwell time for narrow bandwith to more easily see the spectrum shape
    t_end=0.1
)

# create dashboard app
exp3_app = SEApp(
    override_pars=override_pars,
    show_magnitude=True,
    show_complex=False,
    enable_run_loop=True,
    flat_filter=True
)

exp3_app.plot1.figure.height=400
exp3_app.plot2.figure.height=400

# display layout
pn.Column(
    pn.Row(exp3_read_grad_mag, exp3_read_grad_dir, exp3_read_x_grad, exp3_read_y_grad),
    exp3_app.main(),
    sizing_mode='stretch_width'
)

## 4. 2D Imaging

### *k*-Space in 1D

<figure style="float: right;">
<img src="Images/kspaceTrajectory.png" width="500">
<figcaption style="width: 500px;">Figure 4.1: $k_z$ trajectory in a Spin Echo pulse sequence with a constant $z$ shim offset</figcaption>
</figure>

The signal we acquire must generally be Fourier transformed to produce an image, and in order to understand how the pulse sequence and acquisition samples relate to the image obtained it is helpful to use the concept of *k*-space. A *k*-space image is the inverse Fourier transform of the image in physical space (the "normal" image). The value of a pixel in a *k*-space image represents the amount of a particular *spatial frequency* (units of radians per meter) in the physical space image, with the centre of *k*-space corresponding to low spatial frequencies and the edges to high spatial frequencies.

Every discrete sample acquired during the pulse sequence has a *k*-space coordinate depending on the gradients and RF pulses applied so far. For frequency encoding as used in the previous sections, $k_z$ evolves as shown in Figure 4.1. The $180^\circ$ RF pulse inverts the accumulated phase of nuclei that were excited by the initial $90^\circ$ pulse, which also inverts the *k*-space coordinate. The pulse sequence is designed so that after the $180^\circ$ pulse $k_z$ is negative and passes through zero at $T_E$ before becoming positive. In this way we sample along a line segment through *k*-space centred at $k_z=0$.

### *k*-Space in 2D

<figure style="float: right;">
<img src="Images/ReadEncoding.png" width="600">
<figcaption style="width: 600px;">Figure 4.2: Projection Reconstruction $k$-space sampling</figcaption>
</figure>

Sampling along a line segment in *k*-space produces a 1D image projected onto a line segment in physical space, and in **Section 3** it was possible to build a rough 2D image by combining multiple 1D projections. In *k*-space, the data acquired lies on line segments through the origin of 2D k-space as shown in Figure 4.2. In principle, instead of processing each line individually with a 1D Fourier transform, we can generalise and apply a 2D Fourier transform to our 2D *k*-space data. In practice this is difficult to do with the data from these line segments because they do not fall in a regular grid, and the Fast Fourier Transform algorithm cannot be used directly. There are advanced data processing techniques that can resample the data into a grid, but these will not be covered in this lab.

Instead, we will modify the pulse sequence as shown in Figure 4.3 to cause the *k*-space coordinates to fall into a regular grid. This is done using *phase encoding*, where a gradient pulse is applied before sampling, causing the accumulated phase of precession to change depending on position. After a $y$ gradient pulse with amplitude $G_y$ and duration $t_y$ the final phase at coordinate $y$ is:

$$ \phi = \bar\gamma G_y t_y \cdot y$$

In this way, the $y$ position can be determined from the phase of the signal of a small partition of sample. With a large sample the signals from different $y$ locations will sum together into a single phase and magnitude, and multiple acquisitions stepping through values of $G_y t_y$ are required to fill the *k*-space grid as shown in Figure 4.4, so that the Fourier transform can be used to separate out the signals from different locations.

<div style="float: left;">
<figure style="float: left;">
<img src="Images/SpinEchoPhase.png" width="500">
<figcaption style="width: 500px;">Figure 4.3: 2D Spin Echo imaging pulse sequence</figcaption>
</figure>

<figure style="float: left;">
<img src="Images/KSpace_spinEcho.png" width="400">
<figcaption style="width: 400px;">Figure 4.4: $k$-space trajectory of a single Spin Echo acquisition</figcaption>
</figure>
</div>

<figure style="float: right;">
<img src="Images/se_parameters.png" width="500">
<figcaption style="width: 500px;">Figure 4.5: Parameters of the 2D Spin Echo imaging experiment</figcaption>
</figure>

> ---
> #### **Task 4.1**
> 1. Insert the shim sample at the correct depth.
> 2. Set the read and phase gradient values to a non-zero value and run the experiment.
> 3. As the experiment progresses, k-space will be filled one line at a time, and should end with a spectral pattern.
> 4. By modifying the gradient values, adjust the FOV so the image fills the plot on the right without being cropped and has the correct aspect ratio (the shim sample is circular). Save the Image plot and record the gradient values used. Note: you can abort the experiment early while trying out values to save time.
> 5. Try increasing the read gradient to maximum. **Question**: what happens to the image?
> 6. Reset the read gradient to the value found in step 4, and try increasing the phase gradient to maximum. **Question**: what happens to the image now? Is it the same as when the read gradient was set too high?
> 7. Insert the mystery phantom at the correct depth and run the experiment with the optimised gradient values. Save the Image plot. **Question**: Does the image match what you drew in **Task 3.1**?
> 8. Adjust the parameters to achieve an image with the same FOV but twice as fine resolution. Save the Image plot. *Hint: Refer back to the Image Characteristics section, and to Figure 4.5 on the right.*
> ---

In [ ]:
# Experiment 4: 2D SE with adjustable X read gradient and Y phase gradient showing kspace and image
read_x_grad = pn.widgets.FloatInput(name='Read X Gradient', start=-1, end=1, step=0.01, value=0, width=200)
samples = pn.widgets.IntInput(name='Read Samples', start=1, end=200, step=1, value=32, width=200)
dwell_time = pn.widgets.IntInput(name='Dwell Time (us)', start=2, end=80, step=2, value=20, width=200)
phase_y_grad = pn.widgets.FloatInput(name='Phase Y Gradient', start=-1, end=1, step=0.01, value=0, width=200)
phase_steps = pn.widgets.IntInput(name='Phase Steps', start=1, end=200, step=1, value=32, width=200)
phase_y_width = pn.widgets.IntInput(name='Duration (us)', start=10, end=1000, step=10, value=200, width=200)

def get_phase_order():
    g_phase_prop = np.linspace(1, -1, phase_steps.value, endpoint=False)
    phase_order = np.argsort(np.abs(g_phase_prop-1e-6))
    return phase_order

def get_g_phase():
    g_phase_max = phase_y_grad.value
    g_phase_prop = np.linspace(1, -1, phase_steps.value, endpoint=False)
    return np.outer(phase_y_grad.value*g_phase_prop[get_phase_order()], np.array([0, 1, 0]))


import importlib
import RARE2D
importlib.reload(RARE2D)

from RARE2D import RARE2DApp # from dashboards-inline directory that was added to sys.path
# set some parameters directly
override_pars = dict(
    g_read=lambda: (read_x_grad.value, 0, 0),
    g_phase_1=get_g_phase,
    phase_order=get_phase_order,
    n_ETL=1,
    t_echo=0.01,
    n_scans=1,
    n_samples=samples,
    t_dw=lambda: 1e-6*dwell_time.value,
    t_read=lambda: 1e-6*dwell_time.value*samples.value,
    t_phase=lambda: 1e-6*phase_y_width.value,
    t_end=1,
)

# create dashboard app
exp4_app = RARE2DApp(
    override_pars=override_pars
)

# display layout
pn.Column(
    pn.Row(read_x_grad, samples, dwell_time),
    pn.Row(phase_y_grad, phase_steps, phase_y_width),
    exp4_app.main(),
    sizing_mode='stretch_width'
)